In [ ]:
# Reverse Geocoding 
from urllib.request import urlopen
import json

def getplace(x):
    
    key = ""
    url = "https://maps.googleapis.com/maps/api/geocode/json?"
    url += "latlng=%s,%s&sensor=false&key=%s" % (x[0], x[1], key)
    country = town = None
    v = urlopen(url).read()
    j = json.loads(v)
    try:
        components = j['results'][1]['address_components']
        for c in components:
            if "country" in c['types']:
                country = c['long_name']
            if "locality" in c['types']:
                town = c['long_name']
    except:
        country = town = 'Undefined'
    return town, country

data = train_data.select('latitude', 'longitude').toPandas()

lat_long = data.groupby(['latitude','longitude']).size().reset_index().rename(columns={0:'count'})

lat_long = lat_long.drop(['count'], axis= 1)

lat_long['town', 'country'] = lat_long.apply(getplace, axis=1)

lat_long.columns = ['latitude', 'longitude', 'location']

lat_long['town'] = lat_long['location'].str[0]
lat_long['country'] = lat_long['location'].str[1]

lat_long.to_csv("../modeled_data/lat_long.csv", index=False)

lat_long = spark.read.format('csv').option('header', 'true').load('../modeled_data/lat_long.csv')

lat_long.dtypes

float_columns = ['latitude', 'longitude']
string_columns = ['town', 'country']

lat_long = lat_long.select(*(col(c).cast("float").alias(c) for c in float_columns), \
                                                 *(col(c).alias(c) for c in string_columns))

lat_long.dtypes

full_data = train_data.join(lat_long, (train_data.latitude == lat_long.latitude) & (train_data.longitude == lat_long.longitude), 'inner')



full_data = full_data.toPandas()
full_data = full_data.fillna(full_data['town'].value_counts().index[0])

full_data.toPandas().to_csv("../modeled_data/train_with_geo.csv", index=False)



In [ ]:
# Categories Map
from pyspark.sql.functions import col,desc
categories_data = train_data.toPandas()

categ

category = pd.read_csv('../modeled_data/categories_ldata.csv')

games = list(category.Games)
finance = list(category.Finance)
ecommerce = list(category.Ecommerce)
social = list(category.Social)
industry = list(category.Industry)
travel = list(category.Travel)
phone = list(category.Phone)
utilities = list(category.Utilities)
stock = list(category.Stock)

fill_data = train_data.toPandas()

def reformat_data(x):
    if x in games:
        x = 'games'
    elif x in finance:
        x = 'finance'
    elif x in ecommerce:
        x = 'ecommerce'
    elif x in social:
        x = 'social'
    elif x in industry:
        x = 'industry'
    elif x in travel:
        x = 'travel'
    elif x in phone:
        x = 'phone'
    elif x in utilities:
        x = 'utilities'
    elif x in stock:
        x = 'stock'
    else:
        x = 'other'
    
    return x

fill_data['category_mapped'] = fill_data['category'].map(reformat_data)

fill_data.category_mapped.unique()